In [1]:
from binary_param import linear, constant
import pandas as pd
import numpy as np
from R125_bmimPF6_PR import configuration
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           units as pyunits)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.unit_models import Flash
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
def PR_model(data):
    """
    Function which returns initilized model.

    Args:
        data: pandas DataFrame with data

    Returns:
        initialized model
    """
    eps = 0.1
    init_kappa_2_1A = 1.2354
    init_kappa_1_2A = 0.1695
    init_kappa_2_1B = -1.387
    init_kappa_1_2B = -0.2075
    
#     m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(-1.195988)
# m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(-0.08814)
# m.fs.properties.PR_kappa_B['bmimPF6', 'R125'].fix(1.56874)
# m.fs.properties.PR_kappa_B['R125', 'bmimPF6'].fix(0.08489)
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})
    x = float(data['x_R125'])+eps
    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.mole_frac_comp['bmimPF6'].fix(1-x)
    m.fs.state_block.mole_frac_comp['R125'].fix(x)

    # parameter - kappa_A_ij (set at 0.3, 0 if i=j)
    m.fs.properties.PR_kappa_A['bmimPF6','bmimPF6'].fix(0)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(init_kappa_2_1A)
    m.fs.properties.PR_kappa_A['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(init_kappa_1_2A)
    m.fs.properties.PR_kappa_B['bmimPF6', 'bmimPF6'].fix(0)
    m.fs.properties.PR_kappa_B['bmimPF6', 'R125'].fix(init_kappa_2_1B)
    m.fs.properties.PR_kappa_B['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_B['R125', 'bmimPF6'].fix(init_kappa_1_2B)
    m.fs.properties.PR_kappa_C['bmimPF6', 'bmimPF6'].fix(0)
    m.fs.properties.PR_kappa_C['bmimPF6', 'R125'].fix(0)
    m.fs.properties.PR_kappa_C['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_C['R125', 'bmimPF6'].fix(0)
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix the state variables on the state block
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp['bmimPF6'].unfix()
    m.fs.state_block.mole_frac_comp['R125'].unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data['x_R125']))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimPF6'].fix(float(data['x_bmimPF6']))
    m.fs.state_block.mole_frac_comp['R125'].fix(float(data['x_R125'])+eps)
    m.fs.state_block.mole_frac_comp['bmimPF6'].unfix()

    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].setlb(-5)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].setub(5)

    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].setlb(-5)
    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].setub(5)
    
    m.fs.properties.PR_kappa_B['bmimPF6', 'R125'].setlb(-5)
    m.fs.properties.PR_kappa_B['bmimPF6', 'R125'].setub(5)

    m.fs.properties.PR_kappa_B['R125', 'bmimPF6'].setlb(-5)
    m.fs.properties.PR_kappa_B['R125', 'bmimPF6'].setub(5)

    # Return initialized flash model
    return m

def SSE(m, data):
    """
    returns objective function expresion.

    Args:
        m: model
        data: pandas DataFrame with data

    Returns:
        objectuve function scaled expresion
    """
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)

    return expr * 1e-6

variable_name = ["fs.properties.PR_kappa_A['bmimPF6', 'R125']",
                 "fs.properties.PR_kappa_A['R125', 'bmimPF6']",
                 "fs.properties.PR_kappa_B['bmimPF6', 'R125']",
                 "fs.properties.PR_kappa_B['R125', 'bmimPF6']"]

#     pest = parmest.Estimator(PR_model, data, variable_name, SSE, tee=True)

#     obj_value, parameters,a= pest.theta_est(calc_cov=True)

In [3]:
data = pd.read_csv('R125IL.csv')
print(data)

    temperature  pressure  x_R125  x_bmimPF6
0         283.1      9900   0.013      0.987
1         283.2     99600   0.034      0.966
2         283.3    199600   0.075      0.925
3         283.2    299600   0.126      0.874
4         283.3    399600   0.185      0.815
5         283.1    499700   0.255      0.745
6         283.2    599700   0.342      0.658
7         283.1    699600   0.462      0.538
8         298.2     99600   0.023      0.977
9         298.2    199800   0.050      0.950
10        298.1    299500   0.080      0.920
11        298.2    399800   0.111      0.889
12        298.2    499500   0.143      0.857
13        298.2    599500   0.180      0.820
14        298.2    799600   0.261      0.739
15        298.1    999800   0.363      0.637
16        323.1     99600   0.013      0.987
17        323.1    199800   0.028      0.972
18        323.2    300000   0.042      0.958
19        323.1    399700   0.057      0.943
20        323.2    499900   0.072      0.928
21        

In [4]:
N = 10 #number of samples
results_params = np.zeros((N,4))
results_obj = np.zeros((N,1))

st_dev_T = 0.01 #C
st_dev_P = 9771 #MPa
st_dev_x = 0.10 #mole fraction

for i in range(N):
    print(i)
#     noise_T = np.random.normal(loc=0,scale=st_dev_T)
#     print(noise_T)
#     noise_P = np.random.normal(loc=0,scale=st_dev_P)
#     print(noise_P)
    noise_x = np.random.normal(loc=0,scale=st_dev_x)
    print(noise_x)
    data_ = data.copy()
#     data_["temperature"] += noise_T
    data_["x_bmimPF6"] += data_["x_bmimPF6"] * noise_x
#     data_["pressure"] += noise_P
    
#     data_.loc[data_['pressure'] <= 0, 'pressure'] = 1000
    data_.loc[data_['x_bmimPF6'] <= 0.0001, 'x_bmimPF6'] = 0.0001
    data_.loc[data_['x_bmimPF6'] >= 0.9999 , 'x_bmimPF6'] = 0.9999      

    try:
        pest = parmest.Estimator(PR_model, data_, variable_name, SSE, tee=True)

        obj_value, parameters = pest.theta_est()

        for k,v in parameters.items():
            print(k, "=", v)

        line = 0

        for k,v in parameters.items():
            results_params[i,line] = v
            line += 1
        results_obj[i] = obj_value

    except ValueError:
        results_params[i,0] = 'NaN'
        results_params[i,1] = 'NaN'


0
0.0351621924250684
2021-12-30 10:37:39 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 10:37:40 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 10:37:41 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 10:37:41 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 10:37:42 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 10:37:43 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 10:37:44 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 10:37:45 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasibl

  28  7.7886451e+04 1.49e+00 5.19e+06  -1.0 1.61e+06    -  1.00e+00 1.94e-02h  5
  29  7.9899524e+04 1.46e+00 5.07e+06  -1.0 1.57e+06    -  1.00e+00 2.15e-02h  5
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  30  8.2733913e+04 1.42e+00 4.93e+06  -1.0 1.51e+06    -  1.00e+00 2.82e-02h  5
  31  2.4902826e+05 3.13e+00 7.20e+06  -1.0 1.44e+06    -  1.00e+00 7.05e-01w  1
  32  2.4902833e+05 3.13e+00 7.20e+06  -1.0 3.06e+04   0.4 2.84e-03 2.99e-05w  1
  33  2.4903966e+05 3.11e+00 7.16e+06  -1.0 2.26e+04  -0.1 6.65e-03 5.62e-03w  1
  34  8.5052731e+04 1.38e+00 4.82e+06  -1.0 1.21e+04  -0.5 1.00e+00 2.20e-02h  5
  35  8.7550791e+04 1.35e+00 4.71e+06  -1.0 1.39e+06    -  1.00e+00 2.29e-02h  6
  36  9.0155721e+04 1.32e+00 4.60e+06  -1.0 1.33e+06    -  1.00e+00 2.32e-02h  6
  37  9.2851108e+04 1.29e+00 4.49e+06  -1.0 1.28e+06    -  1.00e+00 2.35e-02h  6
  38  9.5623063e+04 1.26e+00 4.38e+06  -1.0 1.22e+06    -  1.00e+00 2.38e-02h  6
  39  9.8459147e+04 1.23e+00

 107  2.8046040e+06 1.97e+02 4.52e+11  -2.5 8.33e+08    -  1.01e-02 5.10e-05h  4
 108  2.8531962e+06 1.97e+02 4.52e+11  -2.5 8.33e+08    -  1.01e-02 1.19e-04h  6
 109  2.8907645e+06 1.97e+02 4.52e+11  -2.5 8.32e+08    -  1.03e-02 9.13e-05h  7
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 110  2.9279032e+06 1.97e+02 4.52e+11  -2.5 8.31e+08    -  2.23e-02 8.94e-05h  7
 111  2.9462096e+06 1.97e+02 4.52e+11  -2.5 8.31e+08    -  5.10e-02 4.38e-05h  8
 112  6.0278653e+06 1.95e+02 4.62e+11  -2.5 8.31e+08    -  1.32e-02 5.54e-03w  1
 113  6.0096506e+06 2.56e+02 4.57e+11  -2.5 7.23e+06  -0.8 6.92e-02 1.06e-02w  1
 114  5.9594378e+06 2.58e+02 4.57e+11  -2.5 1.26e+07  -1.3 7.67e-03 1.64e-02w  1
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
 115  2.9552996e+06 1.97e+02 4.52e+11  -2.5 4.85e+06  -0.8 1.32e-02 2.17e-05h  8
 116r 2.9552996e+06 1.97e+02 9.99e+02   2.3 0.00e+00    -  0.00e+00 3.37e-07R 15
 117r 2.9552997e+06 1.67e+02 9

1
0.09490053952967874
2021-12-30 10:38:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 10:38:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 10:38:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 10:38:24 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 10:38:25 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 10:38:26 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 10:38:27 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 10:38:28 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasib

2021-12-30 10:38:59 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 10:39:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 10:39:01 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 10:39:01 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 10:39:02 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 10:39:03 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 10:39:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-12-30 10:39:05 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may 

...

2797  1.0673607e+02 2.62e+00 3.88e+02  -1.7 6.51e+04    -  1.00e+00 1.25e-01h  4
2798  1.0749755e+02 2.53e+00 3.76e+02  -1.7 3.41e+04    -  1.00e+00 3.12e-02h  6
2799  1.0831363e+02 2.45e+00 3.64e+02  -1.7 3.12e+04    -  1.00e+00 3.12e-02h  6
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2800  1.9535117e+02 2.30e+00 4.96e+01  -1.7 2.87e+04    -  1.00e+00 1.00e+00h  1
2801  1.3442115e+02 1.71e+00 1.54e+02  -1.7 3.10e+04    -  1.00e+00 5.00e-01f  2
2802  1.4966703e+02 1.33e+00 1.26e+02  -1.7 6.17e+03    -  1.00e+00 1.00e+00h  1
2803  1.1412128e+02 2.28e+00 4.36e+02  -1.7 1.86e+04    -  1.00e+00 1.00e+00f  1
2804  1.3923975e+02 1.23e+00 5.14e+02  -1.7 2.55e+04    -  1.00e+00 5.00e-01h  2
2805  1.0885921e+02 1.67e+00 2.21e+02  -1.7 1.03e+04    -  1.00e+00 1.00e+00f  1
2806  1.3170320e+02 9.92e-01 2.64e+02  -1.7 2.08e+04    -  1.00e+00 5.00e-01h  2
2807  1.0863519e+02 1.64e+00 1.69e+02  -1.7 1.00e+04    -  1.00e+00 1.00e+00f  1
2808  1.3150042e+02 9.80e-01

2891  1.3144400e+02 9.77e-01 2.35e+02  -1.7 2.00e+04    -  1.00e+00 5.00e-01h  2
2892  1.0856144e+02 1.64e+00 1.64e+02  -1.7 9.98e+03    -  1.00e+00 1.00e+00f  1
2893  1.3144400e+02 9.77e-01 2.35e+02  -1.7 2.00e+04    -  1.00e+00 5.00e-01h  2
2894  1.0856144e+02 1.64e+00 1.64e+02  -1.7 9.98e+03    -  1.00e+00 1.00e+00f  1
2895  1.3144400e+02 9.77e-01 2.35e+02  -1.7 2.00e+04    -  1.00e+00 5.00e-01h  2
2896  1.0553321e+02 3.11e+00 4.27e+02  -1.7 9.98e+03    -  1.00e+00 1.00e+00F  1
2897  1.0499742e+02 2.92e+00 4.52e+02  -1.7 1.08e+05    -  1.00e+00 6.25e-02f  5
2898  1.0718673e+02 2.56e+00 4.18e+02  -1.7 6.32e+04    -  1.00e+00 1.25e-01h  4
2899  1.3902551e+02 1.28e+00 2.07e+02  -1.7 3.38e+04    -  1.00e+00 5.00e-01h  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2900  1.4821491e+02 1.14e+00 4.17e+01  -1.7 8.58e+03    -  1.00e+00 1.00e+00h  1
2901  1.1980188e+02 1.13e+00 1.41e+02  -1.7 1.86e+04    -  1.00e+00 5.00e-01f  2
2902  1.3379504e+02 9.91e-01

2989  1.3278979e+02 9.82e-01 1.15e+02  -1.7 1.23e+04    -  1.00e+00 5.00e-01h  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2990  1.0849648e+02 1.68e+00 1.57e+02  -1.7 1.11e+04    -  1.00e+00 1.00e+00f  1
2991  1.3165048e+02 9.82e-01 2.33e+02  -1.7 2.00e+04    -  1.00e+00 5.00e-01h  2
2992  1.2387144e+02 1.04e+00 1.85e+02  -1.7 9.98e+03    -  1.00e+00 2.50e-01f  3
2993  1.3529647e+02 1.04e+00 1.04e+02  -1.7 8.91e+03    -  1.00e+00 5.00e-01h  2
2994  1.1828762e+02 1.14e+00 1.01e+02  -1.7 1.23e+04    -  1.00e+00 5.00e-01f  2
2995  1.3305912e+02 9.79e-01 8.55e+01  -1.7 1.11e+04    -  1.00e+00 5.00e-01h  2
2996  1.0844279e+02 1.69e+00 1.53e+02  -1.7 1.13e+04    -  1.00e+00 1.00e+00f  1
2997  1.3166058e+02 9.82e-01 2.30e+02  -1.7 1.99e+04    -  1.00e+00 5.00e-01h  2
2998  1.0849786e+02 1.64e+00 1.64e+02  -1.7 9.98e+03    -  1.00e+00 1.00e+00f  1
2999  1.3139601e+02 9.74e-01 2.33e+02  -1.7 1.99e+04    -  1.00e+00 5.00e-01h  2
iter    objective    inf_pr 

   5  2.6422466e+02 1.23e+01 8.75e+06  -1.0 2.04e+04    -  1.13e-01 1.00e+00h  1
   6  3.3113838e+02 5.79e+00 1.40e+05  -1.0 8.20e+03    -  9.84e-01 1.00e+00h  1
   7  2.5937149e+02 2.27e+00 1.20e+04  -1.0 1.04e+04    -  9.41e-01 1.00e+00f  1
   8  1.6600368e+02 8.02e-01 2.57e+03  -1.0 1.44e+04    -  9.70e-01 1.00e+00f  1
   9  1.2384894e+02 1.23e+00 6.43e+05  -1.7 1.12e+04    -  9.20e-01 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  1.1517812e+02 2.45e-01 6.35e+03  -1.7 1.68e+04    -  9.90e-01 1.00e+00f  1
  11  1.1240230e+02 1.29e-03 4.75e+01  -1.7 3.76e+03    -  1.00e+00 1.00e+00f  1
  12  1.1230805e+02 1.61e-05 2.08e-02  -2.5 8.55e+02    -  1.00e+00 1.00e+00f  1
  13  1.1230770e+02 2.74e-10 1.10e-06  -5.7 3.25e+00    -  1.00e+00 1.00e+00h  1
  14  1.1230770e+02 1.02e-12 1.96e-11  -8.6 3.09e-03    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 14

                                   (scaled)                 (unscaled)
Objecti

   6  1.6808318e+02 1.75e+00 1.43e+04  -1.0 4.15e+03    -  9.51e-01 7.21e-01f  1
   7  1.5920297e+02 6.62e-01 1.43e+04  -1.0 7.55e+03    -  9.26e-01 1.00e+00h  1
   8  1.5381606e+02 2.12e-01 1.41e+05  -1.0 3.31e+03    -  9.86e-01 1.00e+00f  1
   9  1.5273942e+02 4.20e-02 2.56e+04  -1.0 1.90e+03    -  8.20e-01 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  1.5270227e+02 2.36e-03 2.51e+02  -1.0 1.32e+01  -4.0 9.90e-01 1.00e+00h  1
  11  1.5252169e+02 4.33e-03 4.43e+06  -2.5 8.83e+02    -  5.44e-01 1.00e+00f  1
  12  1.5247426e+02 7.11e-04 9.71e+03  -2.5 1.88e+01  -4.5 9.98e-01 1.00e+00h  1
  13  1.4302940e+02 3.00e-01 9.45e+03  -2.5 5.35e+03    -  2.29e-02 1.00e+00f  1
  14  1.4056260e+02 2.32e-02 5.70e-01  -2.5 4.42e+03    -  1.00e+00 1.00e+00f  1
  15  1.4054783e+02 1.28e-04 4.64e-03  -3.8 2.96e+01    -  1.00e+00 1.00e+00h  1
  16  1.4054773e+02 2.31e-08 1.25e-06  -5.7 1.51e-01    -  1.00e+00 1.00e+00h  1
  17  1.4054773e+02 7.96e-11

In [5]:
print(results_params)

[[        nan         nan  0.          0.        ]
 [-1.08170923 -0.08449155  1.50007726  0.08612656]
 [-2.66223272 -0.09900508  3.13250963  0.09825692]
 [-1.94711354 -0.16356706  2.07232948  0.1409425 ]
 [        nan         nan  0.          0.        ]
 [-0.97205747 -0.0819546   1.25606007  0.07098294]
 [-1.0573743  -0.0838118   1.47616996  0.08554747]
 [-1.59429645 -0.09173743  1.944197    0.08503543]
 [-0.94471642 -0.07982131  1.36607659  0.08203673]
 [-1.24354673 -0.08786082  1.59753518  0.08306918]]


In [6]:
results = pd.DataFrame(results_params)
results.to_csv('R125_bmim_10_T.csv')